# CWRU Experiments
## Imports

In [1]:
import pandas as pd
import numpy as np

import lib.transformers as tf
from sklearn.pipeline import Pipeline

from lib.models import Ganomaly1d, Ganomaly2d, GanomalyFE, GanomalyNet
from lib.visualization import GANomalyBoard, rename_tensorboard_key

from skorch.callbacks import PassthroughScoring
import torch

from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
import tensorflow

In [2]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

## Settings

In [3]:
torch.manual_seed(0)

## Initializing Model(s)

In [36]:
def build_model(model, isize, max_epochs, needs_feature_engineering = False, ngpu = 0, nz = 100, ndf = 64, ngf = 64, nc = 1, batch_size = 32, lr = 0.0002, beta1 = 0.5, beta2 = 0.999, workers = 2):
    
    if not needs_feature_engineering:
        output_model = GanomalyNet(
            model,
            module__isize = isize,
            module__nz=nz,
            module__ndf=ndf,
            module__ngf=ngf,
            module__nc=nc,
            module__ngpu=ngpu,

            device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu',

            criterion=torch.nn.BCELoss,

            optimizer_gen=torch.optim.Adam,
            optimizer_gen__lr=lr,
            optimizer_gen__betas=(beta1, beta2),

            optimizer_dis=torch.optim.Adam,
            optimizer_dis__lr=lr,
            optimizer_dis__betas=(beta1, beta2),

            batch_size=batch_size,
            max_epochs=max_epochs,

            train_split=False,  # not implemented
            iterator_train__shuffle=True,
            iterator_train__num_workers=workers,
            iterator_valid__num_workers=workers,

            callbacks=[
                PassthroughScoring('loss_dis', on_train=True),
                PassthroughScoring('loss_gen', on_train=True),
                PassthroughScoring('loss_gen_fra', on_train=True),
                PassthroughScoring('loss_gen_app', on_train=True),
                PassthroughScoring('loss_gen_lat', on_train=True)  
            ]
        )
    else:
            output_model = GanomalyNet(
            model,
            module__isize = isize,
            module__ngpu=ngpu,

            device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu',

            criterion=torch.nn.BCELoss,

            optimizer_gen=torch.optim.Adam,
            optimizer_gen__lr=lr,
            optimizer_gen__betas=(beta1, beta2),

            optimizer_dis=torch.optim.Adam,
            optimizer_dis__lr=lr,
            optimizer_dis__betas=(beta1, beta2),

            batch_size=batch_size,
            max_epochs=max_epochs,

            train_split=False,  # not implemented
            iterator_train__shuffle=True,
            iterator_train__num_workers=workers,
            iterator_valid__num_workers=workers,

            callbacks=[
                PassthroughScoring('loss_dis', on_train=True),
                PassthroughScoring('loss_gen', on_train=True),
                PassthroughScoring('loss_gen_fra', on_train=True),
                PassthroughScoring('loss_gen_app', on_train=True),
                PassthroughScoring('loss_gen_lat', on_train=True)  
            ]
        )
    
    return output_model

In [37]:
ganomaly1d56 = build_model(Ganomaly1d, 56, 1)
ganomaly1d28 = build_model(Ganomaly1d, 28, 1)

ganomaly2d56 = build_model(Ganomaly2d, 56, 1)
ganomaly2d28 = build_model(Ganomaly2d, 28, 1)

ganomaly_fe = build_model(GanomalyFE, 4, 1, True)

## Data Preparation
Building a pipeline of custom transformers to fetch and preprocess CWRU data.

In [51]:
image_size = 56

selection_pipeline = \
Pipeline(steps=[
                ('DataSelector', tf.DataSelector(columns = ['fanEndData', 'driveEndData'], column_values = {'condition': ['Normal Baseline'], 'sampleRate': [12000]})),
                ('ArrayFlattener', tf.ArrayFlattener()),
                #('ArrayEqualizer', tf.ArrayEqualizer()),
                ('ArrayChunker', tf.ArrayChunker(image_size**2)),
                ('ArrayFlattener2', tf.ArrayFlattener()),
                ('FeatureExtractor', tf.FeatureExtractor(axis=1)),
                ('ArrayReshaper', tf.ArrayReshaper((1, 4, 4)))
               ])

chunked_normal_data = selection_pipeline.transform(cwruData) 

In [52]:
chunked_normal_data.shape

(1078, 1, 4, 4)

Train-test-split of the normal CWRU data.

In [53]:
train, test = train_test_split(chunked_normal_data, train_size = 400, test_size= 400, random_state = 0)

In [54]:
len(chunked_normal_data)

1078

## GridSearchCV

In [55]:
search_parameters = {
    'module_w_fra': list(range(0, 101, 10)),
    'module_w_app': list(range(0, 101, 10)),
    'module_w_lat': list(range(0, 101, 10)),
}

In [56]:
#ganomaly_gs = GridSearchCV(ganomaly, search_parameters, refit=False, cv=4)

In [57]:
#ganomaly_gs.fit(train)

## Training Visualization (best parameters)
Adding a TensorBoard for the visualization of the training.

In [58]:
ganomaly_board = GANomalyBoard(SummaryWriter(), key_mapper = rename_tensorboard_key, close_after_train = False)
ganomaly1d56.callbacks += [ganomaly_board]

ganomaly1d56.callbacks

In [59]:
ganomaly_fe.fit(train)

Re-initializing module because the following parameters were re-set: isize, ngpu.
  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss     dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  ------
      1      0.4233      1.7597          1.1551          0.6034          0.0012        2.1830  4.3816


<class 'lib.models.GanomalyNet'>[initialized](
  module_=GanomalyFE(
    (l_fra): BCELoss()
    (l_app): L1Loss()
    (l_dis): L1Loss()
    (discriminator): DiscriminatorNetFE(
      (features): Sequential(
        (initial-conv-1-4): Conv2d(1, 4, kernel_size=(2, 2), stride=(1, 1), bias=False)
        (pyramid-4-batchnorm): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pyramid-4-relu): LeakyReLU(negative_slope=0.2, inplace=True)
        (final-conv-4-8): Conv2d(4, 8, kernel_size=(2, 2), stride=(1, 1), bias=False)
        (pyramid-8-batchnorm): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pyramid-8-relu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (classifier): Sequential(
        (Sigmoid): Sigmoid()
      )
    )
    (generator): GeneratorNetFE(
      (encoder1): EncoderFE(
        (main): Sequential(
          (initial-conv-1-4): Conv2d(1, 4, kernel_size=(2, 2), stride=(1, 1), bias=Fal

In [ ]:
56 / 2 / 2 

In [ ]:
ganomalyFE.predict(test).mean()

In [ ]:
ganomaly.predict(selection_pipeline.set_params(DataSelector__column_values = {'condition': ['Outer Race Fault'], 'sampleRate': [12000]}).transform(cwruData)[:200]).mean()

In [ ]:
input_data = torch.zeros([32, 1, 3136], dtype=torch.float32)

layer1 = torch.nn.Conv1d(1, 32, 16, stride=4, padding = 7, bias = False)
layer2 = torch.nn.Conv1d(32, 64, 16, stride=4, padding = 7, bias = False)
layer3 = torch.nn.Conv1d(64, 128, 16, stride=4, padding = 7, bias = False)
layer4 = torch.nn.Conv1d(128, 100, 16, stride=2, padding = 0, bias = False)

layer5 = torch.nn.ConvTranspose1d(100, 128, 16, stride=2, output_padding = 1, bias = False)
layer6 = torch.nn.ConvTranspose1d(128, 64, 16, stride=4, padding = 7, output_padding = 2, bias = False)
layer7 = torch.nn.ConvTranspose1d(64, 32, 16, stride=4, padding = 7, output_padding = 2, bias = False)
layer8 = torch.nn.ConvTranspose1d(32, 1, 16, stride=4, padding = 7, output_padding = 2, bias = False)

print(input_data.shape)

output1 = layer1(input_data)
print(output1.shape)

output2 = layer2(output1)
print(output2.shape)

output3 = layer3(output2)
print(output3.shape)

output4 = layer4(output3)
print(output4.shape)

output5 = layer5(output4)
print(output5.shape)

output6 = layer6(output5)
print(output6.shape)

output7 = layer7(output6)
print(output7.shape)

output8 = layer8(output7)
print(output8.shape)





In [ ]:
input_data = torch.zeros([32, 1, 56, 56], dtype=torch.float32)

layer1 = torch.nn.Conv2d(1, 32, 4, stride=2, padding = 1, bias = False)
layer2 = torch.nn.Conv2d(32, 64, 4, stride=2, padding = 1, bias = False)
layer3 = torch.nn.Conv2d(64, 128, 4, stride=2, padding = 1, bias = False)
layer4 = torch.nn.Conv2d(128, 100, 4, stride=1, padding = 0, bias = False)

layer5 = torch.nn.ConvTranspose2d(100, 128, 4, stride=1, padding = 0, bias = False)
layer6 = torch.nn.ConvTranspose2d(128, 64, 4, stride=2, padding = 1, bias = False)
layer7 = torch.nn.ConvTranspose2d(64, 32, 4, stride=2, padding = 1, bias = False)
layer8 = torch.nn.ConvTranspose2d(32, 1, 4, stride=2, padding = 1, bias = False)

print(input_data.shape)

output1 = layer1(input_data)
print(output1.shape)

output2 = layer2(output1)
print(output2.shape)

output3 = layer3(output2)
print(output3.shape)

output4 = layer4(output3)
print(output4.shape)

output5 = layer5(output4)
print(output5.shape)

output6 = layer6(output5)
print(output6.shape)

output7 = layer7(output6)
print(output7.shape)

output8 = layer8(output7)
print(output8.shape)